### Basic Introduction

Market Basket Analysis is one of the key techniques used by large retailers to uncover associations between items. It works by looking for combinations of items that occur together frequently in transactions. To put it another way, it allows retailers to identify relationships between the items that people buy.

Association Rules are widely used to analyze retail basket or transaction data, and are intended to identify strong rules discovered in transaction data using measures of interestingness, based on the concept of strong rules.

#### An example of Association Rules

- Assume there are 100 customers
- 10 of them bought milk, 8 bought butter and 6 bought both of them.
- bought milk => bought butter
- **Support** = P(Milk & Butter) = 6/100 = 0.06
- **Confidence** = support/P(Butter) = 0.06/0.08 = 0.75
- **Lift** = confidence/P(Milk) = 0.75/0.10 = 7.5

##### Note: Here's only the DEMO

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from itertools import combinations
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, select, engine
import pandas.io.sql
import pyodbc
import time

from mlxtend.frequent_patterns import apriori #market basket analysis module
from mlxtend.frequent_patterns import association_rules

In [12]:
# Get data we want by date
def getDataFromSQL(start_time, end_time):
    sql = '''select ***demo*** where YearMonthDay between {} and {}  ***demo***'''.format(start_time,end_time)
    
    engine = create_engine('mssql+pymssql://***demo***')
    return pd.read_sql(sql, engine)
    
# For one-hot encoding
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# min_support for products we want above this support
# metric: with min_threshold showing us the combinations we want
def marketbasket(min_support, metric, min_threshold, start_time, end_time):
    df = getDataFromSQL(start_time, end_time)

    df['ProdName'] = df['ProdName'].str.strip()
    df.dropna(axis=0, subset=['InvoiceID'], inplace=True)
    df['InvoiceID'] = df['InvoiceID'].astype('str')
    
    # One-hot encoding to a new data frame that contains all products as columns
    basket = (df.groupby(['ProdSpecies','InvoiceID', 'ProdName'])['Units']
              .count().unstack().reset_index().fillna(0)
              .set_index('InvoiceID'))

    basket = basket.drop(columns=['ProdSpecies']) # we don't need to use it
    basket_sets = basket.applymap(encode_units) # for each invoice, set including products as 1, others as 0.


    frequent_itemsets = apriori(basket_sets, min_support= min_support, use_colnames=True) # Apriori module
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # Add new columns marking as number of aligning products for each invoice
    
    rules = association_rules(frequent_itemsets, metric = metric, min_threshold = min_threshold) # Customize our result, min_threshold is for the metric we define
    rules.columns = ['Prodlist','Product','Prodlist support','Product Support','Support','Confidence','Lift','Leverage','Conviction']
    result = rules[['Product','Prodlist','Confidence','Support','Lift']] # columns we want 
    
    return result

In [13]:
df = marketbasket(min_support = 0.01, 
                  metric = 'confidence',
                  min_threshold = 0.05, 
                  start_time = '20190401', 
                  end_time = '20190401')

In [18]:
df.sort_values(by = 'Confidence', ascending= False).head(10)

,Product,Prodlist,Confidence,Support,Lift
49,(Apoquel),"(Cerenia Oral, Cytopoint)",0.541333,0.027444,1.793212
1,(4 Way Lepto),(Canine Bios Service Items),0.508772,0.015682,11.985306
38,(Apoquel),"(Convenia, Cerenia Inj Emesis)",0.485207,0.011086,1.607289
44,(Apoquel),"(Cytopoint, Cerenia Inj Emesis)",0.472622,0.022171,1.565602
54,(Cytopoint),"(Convenia, Apoquel)",0.462617,0.013384,1.189839
4,(4 Way Lepto),(High Titer Parvo),0.460630,0.015817,10.851209
48,(Cytopoint),"(Cerenia Oral, Apoquel)",0.460317,0.027444,1.183925
16,(Canine Bios Service Items),(Defensor 3),0.442623,0.010950,14.360009
43,(Cytopoint),"(Apoquel, Cerenia Inj Emesis)",0.436170,0.022171,1.121819
32,(Apoquel),"(Cerenia Oral, Cerenia Inj Emesis)",0.407295,0.018115,1.349198
